# For users who don't have a GPU!

In [1]:
# Set up the environment.
!git clone https://github.com/zyddnys/manga-image-translator
%cd manga-image-translator/
!python -m pip install -r requirements.txt


Cloning into 'manga-image-translator'...
remote: Enumerating objects: 9984, done.
remote: Counting objects: 100% (2128/2128), done.
remote: Compressing objects: 100% (187/187), done.
remote: Total 9984 (delta 1990), reused 1941 (delta 1941), pack-reused 7856 (from 2)
Receiving objects: 100% (9984/9984), 87.00 MiB | 15.09 MiB/s, done.
Resolving deltas: 100% (7003/7003), done.
/content/manga-image-translator
Looking in indexes: https://pypi.org/simple, https://frederik-uni.github.io/manga-image-translator-rust/python/wheels/simple/
     | 2.4 MB 12.0 MB/s 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 kB 3.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 39.6 MB/s eta 0:00:00
  Preparing met

In [2]:
# Create a simple test image using Python to trigger model download
!python -c "from PIL import Image; import numpy as np; img = Image.fromarray(np.ones((100,100,3),dtype=np.uint8)*255); img.save('dummy.jpg'); print('Created dummy.jpg for testing')"

!python -m manga_translator  local -i dummy.jpg --verbose

Created dummy.jpg for testing
2025-09-05 20:11:31.151596: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1757103091.172319    1302 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1757103091.178318    1302 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1757103091.194092    1302 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1757103091.194115    1302 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1757103091.194118    1302 computation_placer.

In [3]:
# --- Cell 1: Installation and Environment Setup ---
%cd /content/manga-image-translator/server

PORT = 8001
import os
import re
import sys

# Declare global variables at the top of their respective sections
global bore_executable_found
global ctranslate2_aligned

# Initialize them before any conditional assignment, if necessary,
# or ensure the global declaration comes first.
bore_executable_found = False
ctranslate2_aligned = False

# Kill anything on this port
print(f"Killing any processes on port {PORT}...")
# Use /bin/bash -c to ensure shell features like || true are correctly interpreted
!/bin/bash -c "fuser -k {PORT}/tcp || true"

# Install bore
print("Downloading bore...")
!wget -q https://github.com/ekzhang/bore/releases/download/v0.6.0/bore-v0.6.0-x86_64-unknown-linux-musl.tar.gz

# Check if the tar.gz file was downloaded
if not os.path.exists("bore-v0.6.0-x86_64-unknown-linux-musl.tar.gz"):
    print("Error: bore tar.gz not found! Please check the download URL or internet connection. Cannot proceed without bore.")
    sys.exit(1) # Exit the cell execution
else:
    print("Extracting bore...")
    !tar -xzf bore-v0.6.0-x86_64-unknown-linux-musl.tar.gz

    # Attempt to find the bore executable
    BORE_PATH = ""
    if os.path.exists("bore-v0.6.0-x86_64-unknown-linux-musl/bore"):
        BORE_PATH = "bore-v0.6.0-x86_64-unknown-linux-musl/bore"
    elif os.path.exists("bore"):
        BORE_PATH = "bore"

    if BORE_PATH:
        print(f"Found bore executable at: {BORE_PATH}")
        !mv {BORE_PATH} /usr/local/bin/bore
        !chmod +x /usr/local/bin/bore
        print("Verifying bore installation:")
        !which bore
        bore_executable_found = True # This assignment is fine after global declaration
    else:
        print("Error: bore executable not found after extraction. Please check the directory contents. Bore tunnel will be skipped.")
        bore_executable_found = False


# --- CUDA Version and CTranslate2 Alignment for T4 GPU ---
print("\nDetecting CUDA version for CTranslate2 alignment...")
cuda_version_output = !nvcc --version 2>&1
cuda_version_match = re.search(r"release (\d+\.\d+)", "".join(cuda_version_output))
cuda_major_minor = None

if cuda_version_match:
    cuda_major_minor = cuda_version_match.group(1)
    print(f"Detected CUDA Version: {cuda_major_minor}")

    ctranslate2_whl_url = None
    if cuda_major_minor.startswith("11.8"):
        ctranslate2_whl_url = "https://opennmt.github.io/CTranslate2/whl/cu118.html"
    elif cuda_major_minor.startswith("12."): # Catch all CUDA 12.x and try the latest available CTranslate2 CUDA 12 wheel
        print(f"No direct CTranslate2 wheel for CUDA {cuda_major_minor} found. Attempting 'cu122' which is the latest CUDA 12.x wheel available.")
        ctranslate2_whl_url = "https://opennmt.github.io/CTranslate2/whl/cu122.html"

    if ctranslate2_whl_url:
        print(f"Attempting to install/upgrade CTranslate2 wheel from: {ctranslate2_whl_url}")
        try:
            !pip uninstall -y ctranslate2
            !pip install --upgrade ctranslate2 -f {ctranslate2_whl_url} --no-cache-dir
            print("CTranslate2 installation aligned (or attempted to align) with detected CUDA version.")
            ctranslate2_aligned = True
        except Exception as e:
            print(f"Failed to install CTranslate2 for CUDA {cuda_major_minor} (tried {ctranslate2_whl_url.split('/')[-1]}): {e}")
            print("CTranslate2 will run on CPU. API will be launched with --use-gpu-limited.")
            ctranslate2_aligned = False
    else:
        print(f"No suitable CTranslate2 wheel URL found for CUDA version {cuda_major_minor}.")
        print("CTranslate2 will run on CPU. API will be launched with --use-gpu-limited.")
        ctranslate2_aligned = False
else:
    print("Could not detect CUDA version. Assuming no CUDA or issues. CTranslate2 will run on CPU. API will be launched with --use-gpu-limited.")
    ctranslate2_aligned = False

print("\n--- Setup Complete. Proceed to the next cell to launch the API and Bore tunnel. ---")

/content/manga-image-translator/server
Killing any processes on port 8001...
Extracting bore...
Found bore executable at: bore
Verifying bore installation:
/usr/local/bin/bore

Detecting CUDA version for CTranslate2 alignment...
Detected CUDA Version: 12.5
No direct CTranslate2 wheel for CUDA 12.5 found. Attempting 'cu122' which is the latest CUDA 12.x wheel available.
Attempting to install/upgrade CTranslate2 wheel from: https://opennmt.github.io/CTranslate2/whl/cu122.html
Found existing installation: ctranslate2 3.24.0
Uninstalling ctranslate2-3.24.0:
  Successfully uninstalled ctranslate2-3.24.0
Looking in links: https://opennmt.github.io/CTranslate2/whl/cu122.html
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.8/38.8 MB 85.4 MB/s eta 0:00:00
CTranslate2 installation aligned (or attempted to align) with detected CUDA version.

--- Setup Complete. Proceed to the next cell to launch the API and Bore tunnel. ---


In [ ]:
# --- Cell 2: Launch API, Bore Tunnel, and Stream Logs ---
import subprocess, time, os, sys

# The global variables 'bore_executable_found' and 'ctranslate2_aligned'
# are now correctly set in Cell 1 and will be accessible here.

print(f"Launching API with GPU mode: {'--use-gpu' if ctranslate2_aligned else '--use-gpu-limited'}")

cmd = [
    "python", "-u", "main.py",  # -u flag forces unbuffered output
    "--use-gpu" if ctranslate2_aligned else "--use-gpu-limited", # Use full GPU if CTranslate2 was aligned
    "--verbose",
    "--host", "0.0.0.0",
    "--port", str(PORT),
    "--ignore-errors",
    "--start-instance"
]
# Start API subprocess with line buffering
api = subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True, bufsize=1)

# --- open bore tunnel ---
public_url = None
bore_proc = None

if bore_executable_found:
    bore_cmd = ["bore", "local", str(PORT), "--to", "bore.pub"]
    # Start bore subprocess with line buffering
    bore_proc = subprocess.Popen(bore_cmd, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True, bufsize=1)
    print("\nStarting bore tunnel. Monitoring API and Bore logs...")
else:
    print("\nSkipping bore tunnel creation as bore executable was not found.")
    print("Monitoring API logs only...")

# Combined log reading loop for API and Bore
t_start_monitoring = time.time()
api_ready = False
bore_ready = False
max_wait_time = 300 # Increased to 5 minutes for initial boot and model downloads

try:
    while (time.time() - t_start_monitoring < max_wait_time) and \
          (api.poll() is None and (bore_proc is None or (bore_proc.poll() is None and not public_url))):

        # Read from API process
        api_line = api.stdout.readline()
        if api_line:
            sys.stdout.write(f"[api] {api_line}")
            # Check for expected API startup lines
            if "Uvicorn running on http://0.0.0.0:" in api_line:
                api_ready = True
            if "Nonce:" in api_line:
                pass
            if "[shared] Running in shared mode" in api_line:
                api_ready = True

        # Read from Bore process if it's running
        if bore_proc:
            bore_line = bore_proc.stdout.readline()
            if bore_line:
                sys.stdout.write(f"[bore] {bore_line}")
                if "listening at bore.pub:" in bore_line.lower():
                    public_port = bore_line.split("bore.pub:")[-1].strip()
                    public_url = f"http://bore.pub:{public_port}"
                    bore_ready = True

        # If both are ready (or bore wasn't started), we can break the initial waiting loop
        if api_ready and (bore_ready or not bore_proc):
            if time.time() - t_start_monitoring > 5 or (api_line is None and bore_line is None):
                break

        # If API process terminated unexpectedly during boot, break and report
        if api.poll() is not None:
            print(f"\n[ERROR] API process terminated unexpectedly during boot with exit code {api.returncode}.")
            break
        # If Bore process terminated unexpectedly during boot, break and report
        if bore_proc and bore_proc.poll() is not None:
            print(f"\n[ERROR] Bore process terminated unexpectedly during boot with exit code {bore_proc.returncode}.")
            break

        # Prevent busy-waiting if no output from either process
        if not api_line and (not bore_proc or not bore_line):
            time.sleep(0.1)

    # After the main waiting loop, print final status
    if public_url:
        print("\n🔓 Public API URL:", public_url)
        print("📜 Docs:", public_url + "/docs")
    elif bore_executable_found:
        print(f"\nFailed to get bore URL within the timeout ({max_wait_time}s).")
    else:
        print("\nBore tunnel creation skipped.")

    if not api_ready:
        print(f"\n[WARNING] API did not report 'ready' within the timeout ({max_wait_time}s). It might still be starting or encountered an issue.")

    # Continue streaming API logs indefinitely or until the cell is interrupted
    print("\nStreaming remaining API logs (Ctrl+C to stop)...")
    while True:
        line = api.stdout.readline()
        if not line:
            # If the API process has terminated, break
            if api.poll() is not None:
                print("API process terminated.")
                break
            time.sleep(0.1) # Prevent busy-waiting
            continue
        sys.stdout.write(f"[api] {line}")

except KeyboardInterrupt:
    print("\nLog streaming stopped by user.")
except Exception as e:
    print(f"\nAn error occurred during log streaming: {e}")
finally:
    # Ensure subprocesses are terminated if the cell is stopped
    print("\nTerminating background processes...")
    if api.poll() is None:
        api.terminate()
        api.wait(timeout=5)
        if api.poll() is None:
            api.kill()
            api.wait(timeout=5)
    if bore_proc and bore_proc.poll() is None:
        bore_proc.terminate()
        bore_proc.wait(timeout=5)
        if bore_proc.poll() is None:
            bore_proc.kill()
            bore_proc.wait(timeout=5)
    print("Background processes terminated.")

# --- block forever so Colab keeps serving ---
print("Keeping Colab session alive (Ctrl+C to interrupt and stop completely)...")
try:
    while True:
        time.sleep(3600)
except KeyboardInterrupt:
    print("Colab session interrupted.")

Launching API with GPU mode: --use-gpu

Starting bore tunnel. Monitoring API and Bore logs...
[api] 2025-09-05 21:01:18.991733: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
[bore] 2025-09-05T21:01:11.597943Z  INFO bore_cli::client: connected to server remote_port=6087
[api] WARNING: All log messages before absl::InitializeLog() is called are written to STDERR
[bore] 2025-09-05T21:01:11.597965Z  INFO bore_cli::client: listening at bore.pub:6087

🔓 Public API URL: http://bore.pub:6087
📜 Docs: http://bore.pub:6087/docs

[WARNING] API did not report 'ready' within the timeout (300s). It might still be starting or encountered an issue.

Streaming remaining API logs (Ctrl+C to stop)...
[api] E0000 00:00:1757106079.012984   13894 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been regi